In [1]:
import os, sys, inspect
current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
folder_parts = current_folder.split(os.sep)
pywikibot_folder = os.sep.join(folder_parts[0:-1])
if current_folder not in sys.path:
    sys.path.insert(0, current_folder)
if pywikibot_folder not in sys.path:
    sys.path.insert(0, pywikibot_folder)

import pywikibot as pb

from openpyxl import Workbook, load_workbook
import hashlib
from StringIO import StringIO
from mako.template import Template
from pywikibot.specialbots import UploadRobot
import re

In [6]:
EXCEL_FILE = "diario.madrid.es.xlsx"
wb = load_workbook(filename = EXCEL_FILE)
commons_site = pb.Site("commons", "commons")

In [7]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Diario de Madrid - ${title}]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Images from Ayuntamiento de Madrid (to classify)]]
[[Category:${month} ${year} in Madrid]]"""

In [8]:
def get_hash (file_path):
    image_file = open(file_path, "rb")
    byte_stream = StringIO(image_file.read()).getvalue()
    image_file.close()
    hashObject = hashlib.sha1()
    hashObject.update(byte_stream)
    return hashObject.hexdigest()

def is_commons_file (sha):
    commons = ''
    pages = pb.site.APISite.allimages(commons_site, sha1=sha)
    result = False
    for page in pages :
        result = True
        break
    return result

def reduce_name (title) :
    if len(title) > 200 :
        title_tokens = title.split(u'(')
        if len (title_tokens) > 2 :
            title_lead = u'('.join([title_tokens[i] for i in range (len(title_tokens)-1)]).strip()
            title_end = title_tokens[-1]
        else :
            title_lead = title_tokens[0].strip()
            title_end = title_tokens[1]

        title_lead = title_lead[:189]
        title_lead_tokens = title_lead.split(u' ')
        title_lead_tokens = [title_lead_tokens[i] for i in range (len(title_lead_tokens)-1)]
        title_lead = u' '.join(title_lead_tokens)
        title_lead = title_lead + u'...'
        title = u'%s (%s' %(title_lead, title_end)
    title = title.replace(u':', u',')
    return title

In [9]:
onlyfiles = [f for f in os.listdir(os.path.join("d:/", "Temp")) if os.path.isfile(os.path.join("d:/", "Temp", f))]
for image in onlyfiles:
    image_path = os.path.join("d:/", "Temp", image)
    sha = get_hash(image_path)
    if is_commons_file (sha) == False:
        print image
    else :
        os.remove(image_path)

.picasa.ini
136 actividades y m�s de 100 entidades descubren el patrimonio m�s secreto de Madrid (02).jpg
136 actividades y m�s de 100 entidades descubren el patrimonio m�s secreto de Madrid (04).jpg
Acuerdo para favorecer la incorporaci�n de personas desempleadas al mercado laboral (01).jpg
Acuerdo para favorecer la incorporaci�n de personas desempleadas al mercado laboral (02).jpg
Acuerdo para favorecer la incorporaci�n de personas desempleadas al mercado laboral (03).jpg
Acuerdo para favorecer la incorporaci�n de personas desempleadas al mercado laboral (04).jpg
Acuerdo para favorecer la incorporaci�n de personas desempleadas al mercado laboral (05).jpg
Acuerdo para favorecer la incorporaci�n de personas desempleadas al mercado laboral (06).jpg
Acuerdo para favorecer la incorporaci�n de personas desempleadas al mercado laboral (07).jpg
Acuerdo para favorecer la incorporaci�n de personas desempleadas al mercado laboral (08).jpg
Acuerdo para favorecer la incorporaci�n de personas dese

In [10]:
ws = wb[u'Sheet']
for row in ws.iter_rows():
    file_path = row[0].value
    name = file_path.split('\\\\')[-1]
    file_path = os.path.join("d:/", "Temp", name)
    if os.path.isfile(file_path) == False :
        continue
    try :
        vars = {
                "url": row[3].value,
                "description": row[7].value,
                "year": row[5].value,
                "month": row[6].value,
                "date": row[4].value,
                "title": row[2].value
                }
        t = Template(template)
        _text = t.render(**vars)

        bot = UploadRobot(file_path,
                          description = _text,
                          useFilename = re.sub(r"([0-9]{1})\/([0-9]{1})", r"\1 de \2", name),
                          keepFilename = True,
                          verifyDescription = False,
                          ignoreWarning = True,
                          targetSite = commons_site)
        bot.run()
        os.remove(file_path)
    except :
        print "File already deleted"
        continue

The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=Manuela Carmena, alcaldesa de Madrid, ha reivindicado hoy la urgencia de construir “el derecho a la paz”. Lo ha hecho en el acto institucional para conmemorar el Día de Naciones Unidas, al que han asistido, entre otros,  Taleb Rifai, secretario general de la Organización Mundial del Turismo (OMT), e Ildefonso Castro López, secretario de Estado de Asuntos Exteriores de España.  “Es discutible para muchos si es o no un derecho pero que es una necesidad no tiene discusión alguna”, recalcó la alcaldesa.}}
|date=2017-10-24
|source=[https://diario.madrid.es/blog/notas-de-prensa/la-alcaldesa-reivindica-la-necesidad-de-construir-un-derecho-a-la-paz/ Diario de Madrid - La alcaldesa reivindica la necesidad de construir “un derecho a la paz”]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Dis